In [2]:
import sqlite3
import pandas as pd

In [3]:
db = sqlite3.connect('Hop_Teaming_2018.sqlite')

In [5]:
with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    sqliteCursor = db.cursor()
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS nppes_npi ON nppes (npi)')
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS taxonomy_code ON taxonomy (code)')
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS display_name ON taxonomy (display_name)')
    db.execute('CREATE INDEX IF NOT EXISTS section ON taxonomy (section)')
    db.execute('CREATE INDEX IF NOT EXISTS nppes_organization_name ON nppes (`provider_organization_name_(legal_business_name)`)')

## First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

To get major hospitals:  
1. join zip_cbsa table (cbsa 94180)
2. file the total number of referrals order by DESC
3. select top 10 / 15 entity with highest number of referrals

EXPLAIN QUERY PLAN - show estimated query run time 

In [16]:
query = '''
SELECT t.specialization, n2.`provider_organization_name_(legal_business_name)`, 
       SUM(hop.transaction_count) AS patients_referral
FROM Hop_Teaming_2018 AS hop
INNER JOIN nppes n1
ON hop.from_npi = n1.npi
INNER JOIN nppes n2
ON hop.to_npi = n2.npi 
INNER JOIN taxonomy as t
ON n1.primary_taxonomy = t.code
WHERE n1.entity_type_code = 1
    AND n2.provider_business_practice_location_address_city_name = 'NASHVILLE'
    AND n2.provider_business_practice_location_address_state_name = 'TN'
    AND n2.entity_type_code = 2
    AND t.specialization <> 'None'
GROUP BY t.specialization, n2.`provider_organization_name_(legal_business_name)`
ORDER BY patients_referral DESC
LIMIT 100;
'''

In [12]:
query = '''
SELECT t.specialization, t.display_name, 
       SUM(hop.transaction_count) AS patients_referral
FROM Hop_Teaming_2018 AS hop
INNER JOIN nppes n1
ON hop.from_npi = n1.npi
INNER JOIN nppes n2
ON hop.to_npi = n2.npi 
INNER JOIN taxonomy as t
ON n1.primary_taxonomy = t.code
WHERE n1.entity_type_code = 1
    AND n2.provider_business_practice_location_address_postal_code LIKE '37%'
    AND n2.entity_type_code = 2
    AND t.specialization <> 'None'
    AND t.display_name LIKE '%Hospital'
    AND t.section = 'Non-Individual'
GROUP BY t.specialization, t.display_name
ORDER BY patients_referral DESC
LIMIT 20;
'''

## Tomo filters added (t.display_name LIKE '%Hospital%', t.section = 'Non-Individual')

In [ ]:
SELECT DISTINCT organization_name
FROM nnpes AS np
INNER JOIN nucc_taxonomy AS nt
ON np.primary_taxonomy = nt.code
WHERE entity_type_code = 2.0
    AND addresS_city_name = 'NASHVILLE'
    AND addresS_state_name = 'TN'
    AND display_name LIKE '%Hospital%'
    AND section = 'Non-Individual'
    
#query = """
#SELECT *
#FROM nnpes AS np
#INNER JOIN nucc_taxonomy AS nt
#ON np.primary_taxonomy = nt.code
#WHERE organization_name LIKE '%TriStar%'
#    AND entity_type_code = 2.0
#    AND address_city_name = 'NASHVILLE'
#    AND address_state_name = 'TN'
#"""

In [9]:
query = '''
SELECT *
FROM nppes
LIMIT 20;
'''

In [17]:
with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    Nashville_sqlite = pd.read_sql(query, db)

In [18]:
Nashville_sqlite

,specialization,provider_organization_name_(legal_business_name),patients_referral
0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",358401
1,Anatomic Pathology & Clinical Pathology,ASSOCIATED PATHOLOGISTS LLC,304988
2,Anatomic Pathology & Clinical Pathology,"PATHGROUP LABS, LLC",212407
3,Diagnostic Radiology,VANDERBILT UNIVERSITY MEDICAL CENTER,201804
4,Diagnostic Radiology,RADIOLOGY ALLIANCE PC,201363
...,...,...,...
95,Anatomic Pathology & Clinical Pathology,COLUMBIA MEDICAL GROUP - THE FRIST CLINIC INC,13657
96,Diagnostic Radiology,"FIRST CALL AMBULANCE SERVICE, LLC",13612
97,Psychiatric/Mental Health,"WELLNESS SOLUTIONS GERIATRICS, PLLC",13527
98,Pulmonary Disease,SAINT THOMAS WEST HOSPITAL,13297
